## Deployment

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
wine = pd.read_csv('../data/white_wine.csv')
wine.head()

fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0            7.0              0.27         0.36            20.7      0.045   
1            6.3              0.30         0.34             1.6      0.049   
2            8.1              0.28         0.40             6.9      0.050   
3            7.2              0.23         0.32             8.5      0.058   
4            7.2              0.23         0.32             8.5      0.058   

   free sulfur dioxide  total sulfur dioxide  density    pH  sulphates  \
0                 45.0                 170.0   1.0010  3.00       0.45   
1                 14.0                 132.0   0.9940  3.30       0.49   
2                 30.0                  97.0   0.9951  3.26       0.44   
3                 47.0                 186.0   0.9956  3.19       0.40   
4                 47.0                 186.0   0.9956  3.19       0.40   

   alcohol  quality  
0      8.8      6.0  
1      9.5      6.0  
2     10.1      6.0  
3      9.9      6.0  
4      9.9      6.0

## Preprocess

In [3]:
wine.isna().sum()/len(wine)*100

fixed acidity           0.000000
volatile acidity        0.000000
citric acid             0.000000
residual sugar          0.000000
chlorides               0.000000
free sulfur dioxide     0.000000
total sulfur dioxide    0.000000
density                 0.000000
pH                      0.192308
sulphates               0.192308
alcohol                 0.192308
quality                 0.192308
dtype: float64

In [4]:
sns.distplot(wine['alcohol'])

C:\Users\R7891\Anaconda3\lib\site-packages\seaborn\distributions.py:2551: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)


<AxesSubplot:xlabel='alcohol', ylabel='Density'>

In [5]:
wine['quality']

0      6.0
1      6.0
2      6.0
3      6.0
4      6.0
      ... 
515    6.0
516    5.0
517    6.0
518    6.0
519    NaN
Name: quality, Length: 520, dtype: float64

In [6]:
wine['label'] = np.where(wine['quality']>6.0,1,0)

In [7]:
wine.dropna(inplace=True)

In [8]:
wine.isna().sum()

fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
label                   0
dtype: int64

In [9]:
wine.head()

fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0            7.0              0.27         0.36            20.7      0.045   
1            6.3              0.30         0.34             1.6      0.049   
2            8.1              0.28         0.40             6.9      0.050   
3            7.2              0.23         0.32             8.5      0.058   
4            7.2              0.23         0.32             8.5      0.058   

   free sulfur dioxide  total sulfur dioxide  density    pH  sulphates  \
0                 45.0                 170.0   1.0010  3.00       0.45   
1                 14.0                 132.0   0.9940  3.30       0.49   
2                 30.0                  97.0   0.9951  3.26       0.44   
3                 47.0                 186.0   0.9956  3.19       0.40   
4                 47.0                 186.0   0.9956  3.19       0.40   

   alcohol  quality  label  
0      8.8      6.0      0  
1      9.5      6.0      0  
2     10.1      6.0      0  
3      9.9      6.0      0  
4      9.9      6.0      0

In [10]:
len(wine)

519

In [11]:
wine_deploy = wine[['alcohol', 'density', 'fixed acidity', 'chlorides', 'label']]

# ini ngga usah ditiru, cuman contoh
wine_train = wine_deploy[:400]
wine_test = wine_deploy[400:]

In [12]:
wine_train['label']

0      0
1      0
2      0
3      0
4      0
      ..
395    0
396    0
397    0
398    0
399    0
Name: label, Length: 400, dtype: int32

## Model Training

In [13]:
# Basic Operations
import pandas as pd
import numpy as np

# ML Models
from sklearn.linear_model import LogisticRegression

# Feature Engineering
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, PolynomialFeatures, KBinsDiscretizer
from sklearn.compose import ColumnTransformer

# Evaluation
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

# Model
import pickle
import joblib

In [14]:
wine_train.head()

alcohol  density  fixed acidity  chlorides  label
0      8.8   1.0010            7.0      0.045      0
1      9.5   0.9940            6.3      0.049      0
2     10.1   0.9951            8.1      0.050      0
3      9.9   0.9956            7.2      0.058      0
4      9.9   0.9956            7.2      0.058      0

In [15]:
poly = PolynomialFeatures(degree=3, include_bias=False)

transformer = ColumnTransformer([
    ('poly', poly, ['alcohol', 'density']),
    ('bin', KBinsDiscretizer(n_bins=3, encode='onehot', strategy='quantile'), ['fixed acidity', 'chlorides'])
])

In [16]:
X = wine_train.drop('label', axis=1)
y = wine_train['label']

X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, stratify=y, random_state=2020)

model = LogisticRegression(random_state=2020)

estimator = Pipeline([
    ('preprocess', transformer),
    ('clf', model)
])

In [17]:
hyperparam_space = {
    'clf__C': [100, 10, 1, 0.1, 0.01, 0.001],
    'clf__solver': ['liblinear', 'newton-cg']
}

skfold = StratifiedKFold(n_splits=5)

grid = GridSearchCV(
    estimator,
    param_grid = hyperparam_space,
    cv = skfold,
    scoring = 'f1',
    n_jobs=-1,
    verbose=1
)

grid.fit(X_train_val, y_train_val)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


C:\Users\R7891\Anaconda3\lib\site-packages\scipy\optimize\linesearch.py:477: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\R7891\Anaconda3\lib\site-packages\scipy\optimize\linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\R7891\Anaconda3\lib\site-packages\scipy\optimize\linesearch.py:477: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\R7891\Anaconda3\lib\site-packages\scipy\optimize\linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
             estimator=Pipeline(steps=[('preprocess',
                                        ColumnTransformer(transformers=[('poly',
                                                                         PolynomialFeatures(degree=3,
                                                                                            include_bias=False),
                                                                         ['alcohol',
                                                                          'density']),
                                                                        ('bin',
                                                                         KBinsDiscretizer(n_bins=3),
                                                                         ['fixed '
                                                                          'acidity',
                                                  

In [19]:
logit_tuned = grid.best_estimator_.fit(X_train_val, y_train_val)

C:\Users\R7891\Anaconda3\lib\site-packages\scipy\optimize\linesearch.py:477: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\R7891\Anaconda3\lib\site-packages\scipy\optimize\linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\R7891\Anaconda3\lib\site-packages\scipy\optimize\linesearch.py:477: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\R7891\Anaconda3\lib\site-packages\scipy\optimize\linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)


In [21]:
from sklearn.metrics import classification_report
print(classification_report(y_test, logit_tuned.predict(X_test)))

              precision    recall  f1-score   support

           0       0.99      0.95      0.97        81
           1       0.82      0.95      0.88        19

    accuracy                           0.95       100
   macro avg       0.90      0.95      0.92       100
weighted avg       0.96      0.95      0.95       100



## Pickle Save

In [22]:
logit_tuned.fit(X,y)

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('poly',
                                                  PolynomialFeatures(degree=3,
                                                                     include_bias=False),
                                                  ['alcohol', 'density']),
                                                 ('bin',
                                                  KBinsDiscretizer(n_bins=3),
                                                  ['fixed acidity',
                                                   'chlorides'])])),
                ('clf',
                 LogisticRegression(C=100, random_state=2020,
                                    solver='newton-cg'))])

In [23]:
filename = 'logit_final.sav'
pickle.dump(logit_tuned, open(filename, 'wb'))

## Pickle Load

In [24]:
loaded_model = pickle.load(open(filename, 'rb'))

In [25]:
loaded_model

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('poly',
                                                  PolynomialFeatures(degree=3,
                                                                     include_bias=False),
                                                  ['alcohol', 'density']),
                                                 ('bin',
                                                  KBinsDiscretizer(n_bins=3),
                                                  ['fixed acidity',
                                                   'chlorides'])])),
                ('clf',
                 LogisticRegression(C=100, random_state=2020,
                                    solver='newton-cg'))])

In [27]:
wine_X = wine_test.drop(columns='label')
wine_y = wine_test['label']

In [28]:
y_pred_test = loaded_model.predict(wine_X)

In [29]:
print(classification_report(wine_y, y_pred_test))

              precision    recall  f1-score   support

           0       1.00      0.98      0.99        98
           1       0.91      1.00      0.95        21

    accuracy                           0.98       119
   macro avg       0.96      0.99      0.97       119
weighted avg       0.98      0.98      0.98       119



## Predict New Observation

In [30]:
X.head()

alcohol  density  fixed acidity  chlorides
0      8.8   1.0010            7.0      0.045
1      9.5   0.9940            6.3      0.049
2     10.1   0.9951            8.1      0.050
3      9.9   0.9956            7.2      0.058
4      9.9   0.9956            7.2      0.058

In [33]:
X.describe()
# deni poin 5

alcohol     density  fixed acidity   chlorides
count  400.000000  400.000000     400.000000  400.000000
mean    10.095250    0.995458       6.820000    0.048332
std      1.069749    0.002584       0.768881    0.020302
min      8.500000    0.990000       5.000000    0.020000
25%      9.200000    0.993600       6.300000    0.038000
50%      9.900000    0.995500       6.800000    0.045000
75%     10.800000    0.997400       7.200000    0.053000
max     12.800000    1.003300      10.200000    0.200000

In [36]:
wine_test.head()

alcohol  density  fixed acidity  chlorides  label
400     11.4   0.9917            6.3      0.037      0
401      8.8   0.9976            6.8      0.044      0
402      9.2   0.9934            6.1      0.069      0
403      8.7   1.0000            7.1      0.046      0
404     12.6   0.9918            6.8      0.035      0

In [37]:
# front-end
alcohol = float(input('Berapa kadar alkoholnya?(8.5 - 12.8): '))
density = float(input('Berapa kepekatannya?(0.99 - 1.0033): '))
fixed_acidity = input('tingkat keasaman?(5.0 - 10.2): ')
chlorides = input('tingkat chlorides?(0.02 - 0.2): ')

#back-end
df = pd.DataFrame([[alcohol, density, fixed_acidity, chlorides]], columns=X.columns, index=[0])
class_label = loaded_model.predict(df)[0]
proba = loaded_model.predict_proba(df)

if class_label == 0:
    print('Low Quality')
else:
    print('High Quality')
    
print(proba)

Berapa kadar alkoholnya?(8.5 - 12.8):  12.6
Berapa kepekatannya?(0.99 - 1.0033):  0.9918
tingkat keasaman?(5.0 - 10.2):  6.8
tingkat chlorides?(0.02 - 0.2):  0.035


Low Quality
[[0.9910432 0.0089568]]
